In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import (GridSearchCV, cross_val_score,
                                     train_test_split)
from sklearn.preprocessing import StandardScaler

In [7]:
DATA_PATH = "https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/data/"
data = pd.read_csv(DATA_PATH + "winequality-white.csv", sep=";")

In [6]:
data[10:150]

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
10,8.1,0.270,0.41,1.45,0.033,11.0,63.0,0.9908,2.99,0.56,12.0,5
11,8.6,0.230,0.40,4.20,0.035,17.0,109.0,0.9947,3.14,0.53,9.7,5
12,7.9,0.180,0.37,1.20,0.040,16.0,75.0,0.9920,3.18,0.63,10.8,5
13,6.6,0.160,0.40,1.50,0.044,48.0,143.0,0.9912,3.54,0.52,12.4,7
14,8.3,0.420,0.62,19.25,0.040,41.0,172.0,1.0002,2.98,0.67,9.7,5
...,...,...,...,...,...,...,...,...,...,...,...,...
145,6.3,0.255,0.37,1.10,0.040,37.0,114.0,0.9905,3.00,0.39,10.9,6
146,5.6,0.160,0.27,1.40,0.044,53.0,168.0,0.9918,3.28,0.37,10.1,6
147,6.4,0.595,0.14,5.20,0.058,15.0,97.0,0.9951,3.38,0.36,9.0,4
148,6.3,0.340,0.33,4.60,0.034,19.0,80.0,0.9917,3.38,0.58,12.0,7


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [16]:
y = data["quality"]
X = data.drop("quality", axis=1)

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.3, random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

In [18]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

LinearRegression()

In [28]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, linreg.predict(X_train_scaled)))
print("Mean squared error (train): %.3f" % mean_squared_error(y_holdout, linreg.predict(X_holdout_scaled)))

Mean squared error (train): 0.558
Mean squared error (train): 0.584


In [34]:
#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
linreg.coef_

array([ 9.78219223e-02, -1.92259947e-01, -1.83224449e-04,  5.38164096e-01,
        8.12724353e-03,  4.21804406e-02,  1.43040227e-02, -6.65720472e-01,
        1.50036006e-01,  6.20533605e-02,  1.29533447e-01])

In [35]:
linreg.rank_

11

In [36]:
linreg.singular_

array([105.02943578,  74.37879206,  64.81841936,  59.04973256,
        57.72025778,  57.11203382,  49.02342135,  44.91146082,
        37.30295321,  31.59526557,   7.26606247])

In [37]:
linreg.intercept_

5.876896149358229

In [38]:
linreg.n_features_in_

11

In [49]:
#Feature this linear regression model treats as the most influential on wine quality

linreg_coef = pd.DataFrame(
    {"coef": linreg.coef_, "coef_abs": np.abs(linreg.coef_)},
    index=data.columns.drop("quality"),
)

linreg_coef.sort_values(by="coef_abs", ascending=False)

,coef,coef_abs
density,-0.665720,0.665720
residual sugar,0.538164,0.538164
volatile acidity,-0.192260,0.192260
pH,0.150036,0.150036
alcohol,0.129533,0.129533
...,...,...
sulphates,0.062053,0.062053
free sulfur dioxide,0.042180,0.042180
total sulfur dioxide,0.014304,0.014304
chlorides,0.008127,0.008127
